# 读取数据

## 提取基差数据并进行差分

In [2]:
import new_factor as nf

myfactor = nf.NewFactor()
df_basis = myfactor.get_data('basis')
df_basis['T_diff'] = df_basis['T'].diff()
df_basis['TF_diff'] = df_basis['TF'].diff()
df_basis['TS_diff'] = df_basis['TS'].diff() 
df_basis['TL_diff'] = df_basis['TL'].diff()

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2024 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


## 读取因子数据

In [ ]:
import tech_factor as tf


In [1]:
import numpy as np
import pandas as pd

df_raw = pd.read_excel('gold.xlsx')
df_raw.set_index('date', inplace=True)

# 计算差分（可选）
df_raw['diff'] = df_raw['收盘价'].diff()

# ============ 技术指标 ============

# 移动均线（MA5, MA20, MA60）
df_raw['ma5'] = df_raw['收盘价'].rolling(window=5).mean()
df_raw['ma20'] = df_raw['收盘价'].rolling(window=20).mean()
df_raw['ma60'] = df_raw['收盘价'].rolling(window=60).mean()

# 也可以加一些常见的技术指标：
# 波动率（5日、20日标准差）
df_raw['volatility5'] = df_raw['收盘价'].rolling(window=5).std()
df_raw['volatility20'] = df_raw['收盘价'].rolling(window=20).std()

# 均线差值（趋势信号）
df_raw['ma5_ma20_diff'] = df_raw['ma5'] - df_raw['ma20']

# ============ 标签 ============

# 涨跌标签：下一期比当前期涨为 1，跌为 -1，不变为 0
df_raw['label'] = (df_raw['收盘价'].shift(-1) - df_raw['收盘价']).apply(
    lambda x: 1 if x > 0 else -1 if x < 0 else 0
)

# 删除最后一行（因为没有未来数据做标签）
df_clean = df_raw.iloc[:-1]

In [2]:
# 假设 df_clean 已经准备好，索引为时间或顺序

label_col = "label"
split_point = int(len(df_clean) * 0.8)

# 按顺序切分
df_train = df_clean.iloc[:split_point]
df_test  = df_clean.iloc[split_point:]

# 拆分特征和标签
X_train, y_train = df_train.drop(columns=[label_col]), df_train[label_col]
X_test,  y_test  = df_test.drop(columns=[label_col]),  df_test[label_col]

train_data = pd.concat([X_train,y_train],axis=1)
test_data = pd.concat([X_test, y_test],axis = 1)

In [3]:
from autogluon.tabular import TabularDataset, TabularPredictor
predictor = TabularPredictor(label=label_col).fit(train_data)

d:\conda\envs\pavane\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No path specified. Models will be saved in: "AutogluonModels\ag-20250924_062624"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.11.13
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          16
Memory Avail:       5.57 GB / 15.73 GB (35.4%)
Disk Space Avail:   46.70 GB / 274.71 GB (17.0%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='extreme' : New in v1.4: Massively better than 'best' o